In [1]:
pip install jupyter-dash


  Using cached jupyter_dash-0.4.0-py3-none-any.whl (20 kB)
  Using cached dash-1.20.0-py3-none-any.whl
  Using cached ansi2html-1.6.0-py3-none-any.whl (14 kB)
  Using cached retrying-1.3.3-py3-none-any.whl
  Using cached plotly-4.14.3-py2.py3-none-any.whl (13.2 MB)
  Using cached dash_renderer-1.9.1-py3-none-any.whl
  Using cached dash_core_components-1.16.0-py3-none-any.whl
  Using cached dash_html_components-1.1.3-py3-none-any.whl
  Using cached dash_table-4.11.3-py3-none-any.whl
  Using cached Flask_Compress-1.9.0-py3-none-any.whl (7.3 kB)
  Using cached Brotli-1.0.9-cp38-cp38-macosx_10_9_x86_64.whl (421 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install plotly


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import base64
import datetime
import io
from collections import OrderedDict

import plotly.graph_objs as go
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
import plotly.express as px
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.config['suppress_callback_exceptions'] = True

app.layout = html.Div([
    html.Div([
        html.H1("Step 1: Select your File"),
        dcc.Upload(
            id='upload_file',
            children=html.Div([
                html.A('Click to upload file')
            ]),
            style={
                'width': '100%',
                'border': 'solid black 1px',
                'textAlign': 'center',
                'background': '#52527a'
            },
            multiple=False
        ),
        # This is where the button and the file name get displayed when the file is uploaded correctly
        html.Br(),
        html.Div(id='data_uploaded')
    ],
        style={
            'padding': '5%',
            'border': 'solid black 1px',
            'text-align': 'center'
        }
    ),
    # This is where the table will get displayed when the button is clicked
    html.Br(),
    html.Div(
        id='curve_table',
        style={
            'border': 'solid black 1px',
        }
    ),

    # This where the data will be previewed
    html.Div(id='preview_data_table')
])


# Parse data from the uploaded file and the name of the file
def parse_data(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)

    try:
        if "csv" in filename:
            # Assume that the user uploaded a CSV or TXT file
            df = pd.read_csv(io.StringIO(decoded.decode("utf-8")))
        elif "xls" in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        elif "txt" or "tsv" in filename:
            # Assume that the user upl, delimiter = r'\s+'oaded an excel file
            df = pd.read_csv(io.StringIO(decoded.decode("utf-8")), delimiter=r"\s+")
    except Exception as e:
        print(e)
        return html.Div(["There was an error processing this file."])

    # records = df.to_dict('records')
    # print("records", records)
    table1 = html.Div([
        html.Div([
            html.H6(filename + ' uploaded successfully', style={'color': '#009933'}),
            html.Br(),
            ],
            style={
                'text-align': 'center'
            }
        ),

        html.Div([
            html.Button(
                id='btn_parse_data',
                children='Parse Data',
                n_clicks=0,
                style={
                    'background': '#52527a',
                    'color': 'white'
                },
            ),
            html.Br()
        ],
            style={
                'width': '99%',
                'textAlign': 'center'
            }
        )
    ],
        style={
            'text-align': 'center'
        }
    )
    return table1


# Getting the column names for step 2 data table
# Ignore column has been substituted with row_deletable attribute
def update_data_type_table(contents):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

    dropdown_df = pd.DataFrame(OrderedDict([
        ('Column-Name', df.columns),
        ('Data-Type', 'Data type'),
        # ('Ignore?', 'Ignore')
    ]))

    curve_table = html.Div([
        html.H1('Step 2: Describe your data'),
        html.Div([
            dt.DataTable(
                id='curve-table',
                data=dropdown_df.to_dict('record'),
                columns=[
                    {'id': 'Column-Name', 'name': 'Column-Name'},
                    {'id': 'Data-Type', 'name': 'Data-Type', 'presentation': 'dropdown', 'editable': True},
                    {'id': 'Ignore', 'name': 'Ignore?', 'presentation': 'dropdown', 'editable': True}
                ],
                dropdown_conditional=[
                    {
                        'if': {
                            'column_id': 'Data-Type',
                        },
                        'options': [
                            {'label': i, 'value': i}
                            for i in [
                                'Date-Time',
                                'Numerical',
                                'String/Categorical',
                                'Boolean',
                                'Curve Identifier',
                                'Primary Date',
                                'Primary Value'
                            ]
                        ],
                    },
                    {
                        'if': {
                            'column_id': 'Ignore',
                        },
                        'options': [
                            {'label': i, 'value': i}
                            for i in [
                                'True',
                                'no'
                            ]
                        ]
                    }
                ],
                # row_deletable=True
            ),
        ],
            style={
                'width': '99%',
                'color': '#52527a'
            }
        ),

        html.Br(),
        html.Br(),
        html.H1('Step 3: Preview your data'),
        html.H4('Curve identifiers to display'),
        html.Div([
            dcc.Dropdown(
                id='dropdown_curve_val',
                options=[
                    {'label': i, 'value': i} for i in df.Stock.unique()
                ],
                multi=True,
                placeholder='Select value'
            )
        ]),
        html.Br(),
        html.Button(
            id='btn_view_data',
            children='View Data',
            n_clicks=0,
            style={
                'background': '#52527a',
                'color': 'white'
            },
        ),
    ],
        style={
            'padding': '5%',
            'border': 'solid black 1px',
            'text-align': 'center'
        }
    ),

    return curve_table


# Curve identifier
def curve_identifier(contents):

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

    curve_dropdown = html.Div([
        dcc.Dropdown(
            id='dropdown_curve_val',
            options=[
                {'label': i, 'value': i} for i in df.Stock.unique()
            ],
            multi=True,
            placeholder='Select value(s)'
        )
    ])
    return curve_dropdown


# Creating a table with parse data on btn click
def extract_data(contents, data_list, value):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)

    # to get df for the selected stock
    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    # To get all the stocks and y axis values
    df1 = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

    filt = (df['Stock'].isin(value))
    df = df[filt]
    # print('filt', df[filt])
    integer_cols = []
    print(data_list)
    # print('data list', data_list)
    for i in range(0, len(data_list)):
        if data_list[i] == 'Numerical':
            integer_cols.append(data_list[i])

    # print('Integer cols', integer_cols)
    # print("in step 3")
    graph_dt = html.Div([
        # html.H3('Step 3: Preview your data'),
        html.Div([
            html.Br(),
            html.H4('Curve Identifier Table'),
            dt.DataTable(
                id='table_3',
                data=df.to_dict('records'),
                columns=[{'name': i, 'id': i} for i in df.loc[:, data_list]],
                page_size=10
            ),
            html.Br(),
            html.Br(),
            html.H4('Feature Name to Plot'),
            dcc.Dropdown(
                id='plot_data',
                options=[
                    {'label': i, 'value': i} for i in data_list
                ],
                multi=False,
                placeholder='Select Value'
            ),
            html.Br(),
            html.Button(
                id='btn_graph',
                children='View Graph',
                n_clicks=0,
                style={
                    'background': '#52527a',
                    'color': 'white'
                },
            ),
        ],
            style={
                'padding': '5%',
                'border': 'solid black 1px',
                'text-align': 'center'
            }
        ),

        html.Div(id='graph_print')
    ])
    return graph_dt


def print_graph(data, stock_values, y_value, contents, ls_stocks_curr):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)

    filtered_data = []
    for stock in stock_values:
        for dictionary in data:
            if stock in dictionary.values():
                filtered_data.append(dictionary)

    df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

    df = df.loc[df['Stock'].isin(stock_values)]

    # df = pd.DataFrame.from_dict(filtered_data)
    print("in plot graph", df)
    fig = px.scatter(df, x='Date', y=y_value, color='Stock')

    # print("in plot graph", df)
    return html.Div([
        html.H4('Plot of ' + y_value + ' vs Date'),
        html.Br(),
        html.Div([
            dcc.Graph(figure=fig)
        ])
    ],
        style={
            'padding': '5%',
            'border': 'solid black 1px',
            'text-align': 'center'
        }

    )


@app.callback(Output('data_uploaded', 'children'),
              Input('upload_file', 'contents'),
              State('upload_file', 'filename')
              )
def update_file_upload(contents, filename):
    if contents is not None:
        data_extracted = parse_data(contents, filename)

        return data_extracted


@app.callback(Output('curve_table', 'children'),
              Input('btn_parse_data', 'n_clicks'),
              State('upload_file', 'contents')
              )
def parse_data_click(n_clicks, contents):
    if n_clicks != 0:
        curve_table = update_data_type_table(contents)

        return curve_table


# @app.callback(Output('dropdown_curve', 'children'),
#               Input('dropdown_curve_val', 'value'),
#               State('upload_file', 'contents')
#               )
# def show_table_extract_data(value, contents):
#     return 0


@app.callback(Output('preview_data_table', 'children'),
              Input('btn_view_data', 'n_clicks'),
              Input('curve-table', 'data'),
              Input('dropdown_curve_val', 'value'),
              State('upload_file', 'contents')
              )
def show_extract_data(n_clicks, data, value, contents):
    ls_stock = []
    ls_stock_dis = []
    ls_stock_curr = []
    print('drop down val', value)
    # if n_clicks != 0:
    # print('N clicks', n_clicks)
    for d in data:
        for val in d.values():
            ls_stock.append(val)

    # print('ls stock',ls_stock)

    for i in range(0, len(ls_stock)):
        if i % 2 == 0:
            ls_stock_curr.append(ls_stock[i])
        else:
            ls_stock_dis.append(ls_stock[i])
    # table3 = True in (item == 'None' for item in ls_stock)
    # print('table 3',table3)
    # print('ls stock curr', ls_stock_curr)

    if n_clicks != 0:
        table3 = extract_data(contents, ls_stock_curr, value)
        return table3


@app.callback(Output('graph_print', 'children'),
              Input('table_3', 'data'),
              Input('dropdown_curve_val', 'value'),
              Input('plot_data', 'value'),
              Input('btn_graph', 'n_clicks'),
              Input('upload_file', 'contents'),
              State('curve-table', 'data')
              )
def display_final_graph(data, stock_values, y_value, n_click, contents, desc_data):
    print('desc_data', desc_data)

    ls_data = []
    for d in desc_data:
        for vals in d.values():
            ls_data.append(vals)

    ls_stocks_curr = []

    for i in range(0, len(ls_data)):
        if i % 2 == 0:
            ls_stocks_curr.append(ls_data[i])

    print('ls_data', ls_stocks_curr)

    print('Stocks vals', stock_values)
    print('y', y_value)
    print('data', data)
    print('n clicks', n_click)
    # print("in graph callback")
    if n_click != 0:
        graph = print_graph(data, stock_values, y_value, contents, ls_stocks_curr)
        print('Stock values', stock_values)
        return graph


if __name__ == '__main__':
    app.run_server(debug=True , use_reloader= False)

 

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


drop down val None
drop down val ['IBM']
drop down val ['IBM', 'FB']
drop down val ['IBM', 'FB']
['Date', 'Volume', 'Adj Close', 'Stock', 'Exchange']
desc_data [{'Column-Name': 'Date', 'Data-Type': 'Data type'}, {'Column-Name': 'Volume', 'Data-Type': 'Data type'}, {'Column-Name': 'Adj Close', 'Data-Type': 'Data type'}, {'Column-Name': 'Stock', 'Data-Type': 'Data type'}, {'Column-Name': 'Exchange', 'Data-Type': 'Data type'}]
ls_data ['Date', 'Volume', 'Adj Close', 'Stock', 'Exchange']
Stocks vals ['IBM', 'FB']
y None
data [{'Date': '1/2/2020', 'Volume': 3148600, 'Adj Close': 126.9752045, 'Stock': 'IBM', 'Exchange': 'NYSE'}, {'Date': '1/3/2020', 'Volume': 2373700, 'Adj Close': 125.9625397, 'Stock': 'IBM', 'Exchange': 'NYSE'}, {'Date': '1/6/2020', 'Volume': 2425500, 'Adj Close': 125.7375259, 'Stock': 'IBM', 'Exchange': 'NYSE'}, {'Date': '1/7/2020', 'Volume': 3090800, 'Adj Close': 125.821907, 'Stock': 'IBM', 'Exchange': 'NYSE'}, {'Date': '1/8/2020', 'Volume': 4346000, 'Adj Close': 126.8720